In [1]:
import sys
from PIL import Image
sys.modules['Image'] = Image 

In [2]:
from PIL import Image
print(Image.__file__)


C:\Python39\lib\site-packages\PIL\Image.py


In [3]:
import Image
print(Image.__file__)


C:\Python39\lib\site-packages\PIL\Image.py


In [4]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

In [5]:
batch_size=8

In [6]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'C:\Python39\Scripts\Drosiness detection model with MRL dataset\prepared_data\train',
                                              target_size=(80,80),
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              subset='training' )
validation_data= train_datagen.flow_from_directory(r'C:\Python39\Scripts\Drosiness detection model with MRL dataset\prepared_data\test',
                                                   target_size=(80,80),
                                                   batch_size=batch_size,
                                                   class_mode='categorical', 
                                                   subset='validation')


Found 61130 images belonging to 2 classes.
Found 1696 images belonging to 2 classes.


In [7]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_data = test_datagen.flow_from_directory(
    r'C:\Python39\Scripts\Drosiness detection model with MRL dataset\prepared_data\test',
    target_size=(80,80), 
    batch_size = 8, 
    class_mode='categorical')

Found 8487 images belonging to 2 classes.


In [8]:
bmodel = InceptionV3(include_top = False, weights = 'imagenet', 
                     input_tensor = Input(shape = (80,80,3)))

In [9]:
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)
model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [10]:
# model.summary()

In [11]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau


In [12]:
checkpoint = ModelCheckpoint(r'C:\Python39\Scripts\Drosiness detection model with MRL dataset\Models\Model.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [23]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batch_size,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batch_size,
                   callbacks=callbacks,
                    epochs=5)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18532\2572150247.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batch_size,


Epoch 1/5
7641/7641 [==============================] - ETA: 0s - loss: 0.1570 - accuracy: 0.9410
Epoch 1: val_loss did not improve from 0.20098
7641/7641 [==============================] - 648s 84ms/step - loss: 0.1570 - accuracy: 0.9410 - val_loss: 0.2302 - val_accuracy: 0.8992 - lr: 0.0010
Epoch 2/5
7641/7641 [==============================] - ETA: 0s - loss: 0.1550 - accuracy: 0.9403
Epoch 2: val_loss did not improve from 0.20098
7641/7641 [==============================] - 762s 100ms/step - loss: 0.1550 - accuracy: 0.9403 - val_loss: 0.2658 - val_accuracy: 0.8956 - lr: 0.0010
Epoch 3/5
7641/7641 [==============================] - ETA: 0s - loss: 0.1543 - accuracy: 0.9404
Epoch 3: val_loss did not improve from 0.20098
7641/7641 [==============================] - 645s 84ms/step - loss: 0.1543 - accuracy: 0.9404 - val_loss: 0.2437 - val_accuracy: 0.8968 - lr: 0.0010
Epoch 4/5
7641/7641 [==============================] - ETA: 0s - loss: 0.1552 - accuracy: 0.9423
Epoch 4: val_loss did n

In [24]:
loss_tr, acc_tr = model.evaluate(train_data, verbose=1)
print("Training Accuracy: {:.2f}%".format(acc_tr * 100))
print("Training Loss: {:.4f}".format(loss_tr))


7642/7642 [==============================] - 426s 55ms/step - loss: 0.1237 - accuracy: 0.9525
Training Accuracy: 95.25%
Training Loss: 0.1237


In [25]:
loss_vr, acc_vr = model.evaluate_generator(validation_data)
print("Validation Accuracy: {:.2f}%".format(acc_vr * 100))
print("Validation Loss: {:.4f}".format(loss_vr))


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18532\61866403.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  loss_vr, acc_vr = model.evaluate_generator(validation_data)


Validation Accuracy: 90.57%
Validation Loss: 0.2187


In [26]:
loss_test ,acc_test = model.evaluate_generator(test_data)
print("Test Accuracy: {:.2f}%".format(acc_test * 100))
print("Test Loss: {:.4f}".format(loss_test))

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18532\1878360815.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  loss_test ,acc_test = model.evaluate_generator(test_data)


Test Accuracy: 90.57%
Test Loss: 0.3332
